In [31]:
import heapq
import pandas as pd
import random
from datetime import datetime, timedelta
from collections import deque

# --------------------------------------------------------------------------
# DATA CLEANING FUNCTIONS
# --------------------------------------------------------------------------
def remove_outliers_iqr(df, cols):
    for c in cols:
        Q1, Q3 = df[c].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        df = df[df[c].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    return df

def drop_rows_without_true_outfeed(df, prefix="Outfeed"):
    cols = [c for c in df.columns if c.startswith(prefix)]
    return df[df[cols].any(axis=1)] if cols else df

def clean_parcel_data(df):
    df = df.dropna().reset_index(drop=True)
    df = remove_outliers_iqr(df, ["Length", "Width", "Height"])
    df = drop_rows_without_true_outfeed(df)
    return df

def load_parcels_from_clean_df(df):
    parcels = []
    for _, r in df.iterrows():
        parcels.append(Parcel(
            pid=int(r["Parcel Number"]),
            arrival_time=pd.to_datetime(r["Arrival Time"]),
            length=float(r["Length"]),
            width=float(r["Width"]),
            height=float(r["Height"]),
            weight=float(r["Weight"]),
            feasible=[i for i, f in enumerate(
                [r["Outfeed 1"], r["Outfeed 2"], r["Outfeed 3"]]) if f]
        ))
    return sorted(parcels, key=lambda p: p.arrival_time)

# --------------------------------------------------------------------------
# EVENT, FES, PARCEL
# --------------------------------------------------------------------------
class Event:
    ARRIVAL = 0
    ENTER_SCANNER = 1
    ENTER_OUTFEED = 2
    EXIT_OUTFEED = 3
    RECIRCULATE = 4
    def __init__(self, typ, time, parcel, outfeed_id=None): # type is a reserved word
        """
        Parameters:
        param1 (typ): What type of event it is, e.g, Arrival, scanner...
        param2 (time): at what time the event occurs.
        param3 (parcel): all the information of the parcel that is being processed.
        param4 (outfeed_id): the outfeed to which the parcel goes. It is an optional parameter since it is only needed for events 2 and 3.
        """
        self.type = typ
        self.time = time  # float: seconds since t0
        self.parcel = parcel
        self.outfeed_id = outfeed_id # Only used for ENTER_OUTFEED and EXIT_OUTFEED events
    def __lt__(self, other):
        return self.time < other.time

class FES:
    """
    Future Event Set (FES) for discrete event simulation.
    This class uses a priority queue to manage events in the simulation.
    Events are sorted by their time attribute, which is a float.
    """
    def __init__(self):
        self.events = []
    def add(self, event):
        heapq.heappush(self.events, event)
    def next(self):
        return heapq.heappop(self.events)
    def isEmpty(self):
        return len(self.events) == 0

class Parcel: #This replicates the Customer class, in which info about the customers (parcels) is stored.
    def __init__(self, pid, arrival_time, length, width, height, weight, feasible):
        self.id = pid
        self.arrival_time = arrival_time
        self.length = length
        self.width = width
        self.height = height
        self.weight = weight
        self.feasible_outfeeds = feasible
        self.sorted = False
        self.recirculated = False
        self.outfeed_attempts = [] #Afterwards, makes a copy of the feasible outfeeds of the parcel. Used for the algorithm functioning.
        self.recirculation_count = 0

    def get_volume(self):
        return self.length * self.width * self.height

# --------------------------------------------------------------------------
# OUTFEED MODEL
# --------------------------------------------------------------------------
def compute_outfeed_time(parcel):
    base_time = 4.5

    # Volume classes
    volume = parcel.get_volume()
    if volume < 0.035:
        volume_class_delay = 0
    elif volume < 0.055:
        volume_class_delay = 1
    else:
        volume_class_delay = 2

    # Weight classes
    weight = parcel.weight
    if weight < 1700:
        weight_class_delay = 0
    elif weight < 2800:
        weight_class_delay = 1
    else:
        weight_class_delay = 2

    return base_time + volume_class_delay + weight_class_delay

#Probably not to be used in this code, I leave it here now just in case, but the outfeed functioning goes in the simualation class.
class Outfeed:
    def __init__(self, max_length=3.0):
        self.max_length = max_length
        self.current_length = 0.0
        self.queue = []   # list of (Parcel, service_time)
        self.next_time = 0.0
    
    def can_accept(self, parcel):
        return self.current_length + parcel.length <= self.max_length
    
    def add_parcel(self, parcel):
        t = compute_outfeed_time(parcel)
        self.queue.append((parcel, t))
        self.current_length += parcel.length
        if len(self.queue) == 1:
            #Timer for the current parcel 
            self.next_time = t
    
    def update(self, dt):
        self.next_time -= dt
        if self.next_time <= 0 and self.queue:
            p, _ = self.queue.pop(0)
            self.current_length -= p.length
            if self.queue:
                #Timer for the next parcel in line
                self.next_time = self.queue[0][1]

# --------------------------------------------------------------------------
# POSISORTER with Windowed Local Search + Logging + Metrics
# --------------------------------------------------------------------------
class PosiSorterSystem:
    REBALANCE_INTERVAL = 1      # run hill-climb every N arrivals

    def __init__(self, layout_df): #Not sure if i should add arrdist or something similar here.

        L = layout_df.set_index("Layout property")["Value"]
        self.belt_speed = L["Belt Speed"]
        self.d_in_sc = L["Distance Infeeds to Scanner"]
        self.d_sc_of = L["Distance Scanner to Outfeeds"]
        self.d_between = L["Distance between Outfeeds"]
        self.d_of_in = L["Distance Outfeeds to Infeeds"]
        self.num_outfeeds = 3 # Given in Excel sheet. Can be automatically detected from the layout_df if needed.
        self.outfeeds = [Outfeed(max_length=3.0) for _ in range(self.num_outfeeds)]
        #These are used to print statistics about the system:
        self.recirculated_count = 0
        self.outfeed_counts = [0] * self.num_outfeeds
        self.total_processed = 0
        self.first_pass_failures = set()
        # Greedy + periodic windowed local search
        self.loads = {k: 0.0 for k in range(self.num_outfeeds)}   # tracked load per outfeed
        self.assignment = {}                                      # parcel.id -> outfeed or None
        self.WINDOW_DURATION = self.d_sc_of /self.belt_speed      # Time stamp of window in seconds
        self.window = deque()              
        self.rebal_ctr = 0
    
    def greedy(self, p):
        # Initial assignment: choose feasible outfeed with minimal tracked load
        feas = [k for k in p.feasible_outfeeds if self.loads[k] + p.length <= self.outfeeds[k].max_length]
        if not feas:
            return None
        else:
            return min(feas, key=lambda k: self.loads[k])

    def imbalance(self, loads):
        # Objective: load spread between heaviest and lightest outfeed
        return max(loads.values()) - min(loads.values())

    def run_local_search(self, max_iters=100):
        # Hill-climbing to refine assignments in the sliding window
        loads = self.loads.copy()
        assign_w = {}
        # Baseline: assign any unassigned parcels in window greedily
        for _, p in self.window:
            k = self.greedy(p)
            assign_w[p.id] = k  
            if k is not None:
                loads[k] += p.length
        # Iteratively try moves that reduce imbalance
        for _ in range(max_iters):
            improved = False
            for _, p in self.window:
                cur = assign_w[p.id]
                for k in p.feasible_outfeeds:
                    if k == cur or loads[k] + p.length > self.outfeeds[k].max_length:
                        continue
                    new_loads = loads.copy()
                    if cur is not None:
                        new_loads[cur] -= p.length
                    new_loads[k] += p.length
                    if self.imbalance(new_loads) < self.imbalance(loads):
                        loads, assign_w[p.id] = new_loads, k
                        improved = True
                        break
                if improved:
                    break
            if not improved:
                break
        # Commit improved assignments
        for pid, k in assign_w.items():
            self.assignment[pid] = k

    def handle_enter_scanner(self, evt, fes):
        p = evt.parcel
        # 1) greedy assign
        k0 = self.greedy(p)
        self.assignment[p.id] = k0
        # track failure on first pass
        if k0 is None:
            self.first_pass_failures.add(p.id)
        # 2) window and rebalance
        self.window.append((evt.time, p))
        # Remove parcels outside the window
        while self.window and evt.time - self.window[0][0] > self.WINDOW_DURATION:
            self.window.popleft()
        self.rebal_ctr += 1
        if self.rebal_ctr >= self.REBALANCE_INTERVAL:
            self.run_local_search()
            self.rebal_ctr = 0
        # 3) schedule
        t = evt.time
        if self.assignment[p.id] is None:
            # No feasible outfeed: recirculate
            self.recirculated_count += 1
            dt = (self.d_sc_of + self.d_between * self.num_outfeeds) / self.belt_speed
            fes.add(Event(Event.RECIRCULATE, t + dt, p))
        else:
            # Schedule entering chosen outfeed
            k = self.assignment[p.id]
            dt = (self.d_sc_of + k * self.d_between) / self.belt_speed
            fes.add(Event(Event.ENTER_OUTFEED, t + dt, p, outfeed_id=k))

    def simulate(self, parcels):
        fes = FES()
        t = 0
        t0 = parcels[0].arrival_time #We need to convert the arrival time to seconds, since the rest of the times are in seconds.
        #With this for loop, we initiate the simulation by adding all the arrival events of the excel file to the FES. It might not be the most efficient way, 
        # but I think it works since anyways, the events are sorted by time in the FES afterwards.
        for p in parcels:
            t = (p.arrival_time - t0).total_seconds()
            fes.add(Event(Event.ARRIVAL, t, p)) # schedule the event
        while not fes.isEmpty(): # T is still to be determined
            told = t #not being used right now, ususally used to store waiting times.
            evt = fes.next() #retrieve the next event in the FES and removes it
            t = evt.time
            if evt.type == Event.ARRIVAL:
                fes.add(Event(Event.ENTER_SCANNER, evt.time + self.d_in_sc / self.belt_speed, evt.parcel))
            ### Handle outfeed events based on the load balacing and local search
            elif evt.type == Event.ENTER_SCANNER:
                self.handle_enter_scanner(evt, fes)
            elif evt.type == Event.ENTER_OUTFEED:
                k, p = evt.outfeed_id, evt.parcel
                f = self.outfeeds[k]
                f.add_parcel(p)
                self.outfeed_counts[k] += 1
                self.loads[k] += p.length
                if len(f.queue) == 1:
                    fes.add(Event(
                        Event.EXIT_OUTFEED,
                        evt.time + f.queue[0][1],
                        p, outfeed_id=k
                    ))
            elif evt.type == Event.EXIT_OUTFEED:
                k, p = evt.outfeed_id, evt.parcel
                f = self.outfeeds[k]
                f.update(f.next_time)
                self.loads[k] -= p.length
                actual_time = t0 + timedelta(seconds=evt.time)
                print(f"[{actual_time.time()}] Parcel {p.id} removed from outfeed {k}")
                if f.queue:
                    fes.add(Event(
                        Event.EXIT_OUTFEED,
                        evt.time + f.queue[0][1],
                        f.queue[0][0], outfeed_id=k
                    ))
            elif evt.type == Event.RECIRCULATE:
                fes.add(Event(
                    Event.ENTER_SCANNER,
                    evt.time + (self.d_of_in + self.d_in_sc) / self.belt_speed,
                    evt.parcel
                ))
        # final summary
        total = len(parcels)
        sorted_total = sum(self.outfeed_counts)
        success_rate = (total - len(self.first_pass_failures)) / total * 100
        print()
        print(f"Total parcels processed: {len(parcels)}")
        print(f"Parcels recirculated: {self.recirculated_count}")
        print(f"Success rate (no recirc on first pass): {success_rate:.2f}%")
        for i, cnt in enumerate(self.outfeed_counts):
            pct = cnt / sorted_total * 100 if sorted_total > 0 else 0
            print(f"Outfeed {i}: {cnt} parcels, {pct:.2f}% of sorted")
        print(f"Throughput (sorted + recirculated): {sorted_total + self.recirculated_count}")

# ----------------------------------------------------------------------------
# MAIN
# ----------------------------------------------------------------------------
if __name__ == "__main__":
    xls = pd.ExcelFile("PosiSorterData1.xlsx")
    df_p = clean_parcel_data(xls.parse("Parcels"))
    df_l = xls.parse("Layout")
    parcels = load_parcels_from_clean_df(df_p)
    system = PosiSorterSystem(df_l)
    system.simulate(parcels)


[09:00:19.337000] Parcel 1 removed from outfeed 0
[09:00:25.837000] Parcel 2 removed from outfeed 0
[09:00:26.375000] Parcel 16 removed from outfeed 2
[09:00:32.875000] Parcel 149 removed from outfeed 2
[09:00:33.057000] Parcel 3 removed from outfeed 0
[09:00:38.557000] Parcel 4 removed from outfeed 0
[09:00:39.375000] Parcel 133 removed from outfeed 2
[09:00:45.057000] Parcel 5 removed from outfeed 0
[09:00:49.789000] Parcel 6 removed from outfeed 2
[09:00:52.557000] Parcel 7 removed from outfeed 0
[09:00:53.727000] Parcel 9 removed from outfeed 1
[09:00:59.227000] Parcel 10 removed from outfeed 1
[09:01:00.887000] Parcel 11 removed from outfeed 2
[09:01:01.057000] Parcel 8 removed from outfeed 0
[09:01:05.387000] Parcel 12 removed from outfeed 2
[09:01:08.557000] Parcel 14 removed from outfeed 0
[09:01:18.589000] Parcel 15 removed from outfeed 2
[09:01:21.202000] Parcel 19 removed from outfeed 0
[09:01:25.089000] Parcel 17 removed from outfeed 2
[09:01:29.589000] Parcel 18 removed fr